<a href="https://colab.research.google.com/github/aneesh-shetye/NLP/blob/master/sequence_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch 
import torch.nn as nn
import numpy as np
from google.colab import drive
import random

device = torch.device(type='cuda' if torch.cuda.is_available() else 'cpu')
device, torch.cuda.get_device_name(device)

(device(type='cuda'), 'Tesla T4')

In [6]:
!pip install spacy

In [7]:
import spacy
!python -m spacy download en_core_web_md

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [8]:
data = open('charles_dickens_christmas_ghost.txt', 'r').read() 

data = data.lower()
data = data.replace("\n", ".")
data =  data.replace(".", " . ")
data = data.replace(",", " , ")
data =  data.replace(" \' ", " \' ")
data = data.replace("\"", " \" ")

data_train = data.split(".")

data = data.split()
vocab = list(set(data))

vocab_size = len(vocab)

word_to_ix = {ch:i for i,ch in enumerate(vocab)}   
ix_to_word = {i:ch for i,ch in enumerate(vocab)}    

print("No of words in data are: ", len(data))
print("No of word in vocab are: ", len(vocab))
random.shuffle(data)

No of words in data are:  41844
No of word in vocab are:  5516


In [9]:
class lstm_pre(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(lstm_pre, self).__init__()

        self.hidden_size = hidden_size
        self.input_size = input_size      #input_size = vocab_size
        self.num_layers = num_layers

        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers = num_layers)
        self.hn2o = nn.Linear(hidden_size, input_size)
        self.o2o = nn.Linear(hidden_size, input_size)
        self.o2o2 = nn.Linear(input_size, input_size)


    def forward(self, input , hidden, cell_state):

        hidden, (h_n, c_n) = self.lstm1(input, (hidden,cell_state)) #output_shape =[seq_len,batch, num_directions*hidden_size]
        # here we are taking batch = 1 
        output = self.o2o(hidden)
        output = self.o2o2(output)

        return output, h_n, c_n

In [10]:

hidden_size = 100

model_lstm = lstm_pre(input_size=vocab_size, hidden_size=hidden_size, num_layers=1)
model_lstm = model_lstm.to(device)
learning_rate = 0.0005

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=learning_rate)


In [11]:
def train(data_train, hidden_size, vocab_size):
    
    model_lstm.train()

    loss = 0
    no = 0

#training loop over entire data:
    for text in data_train:

        #making initial hidden:
        hidden = torch.zeros([1, 1, hidden_size])
        hidden = hidden.to(device)

        #making initial cell:
        cell = torch.zeros([1, 1, hidden_size])
        cell = cell.to(device)

        text = text.split()

        for i in range(len(text)-1):

            batch_loss = 0

        #making initial input: 
            input = torch.zeros([1, 1, vocab_size])
            input_idx = word_to_ix[text[i]]
            input[0,0,input_idx] = 1
            input = input.to(device)
            

        #making the target tensor:
            target_idx = [word_to_ix[text[i+1]]]
            target = torch.LongTensor(target_idx)
            target = target.to(device)

        #making predictions:

            predictions, h_n , c_n = model_lstm(input, hidden, cell)
            predictions = predictions.to(device)
            h_n = h_n.to(device)
            c_n = c_n.to(device)


        #computing the loss:
            predictions = predictions.view((-1, vocab_size)) #reshaping the predictions(originally [1,1,vocabb_size])
            batch_loss += criterion(predictions, target)
            batch_loss = batch_loss.to(device)

            loss += batch_loss 

            with torch.no_grad():
            #updating hidden state and cell state:  
                hidden = h_n.clone()
                hidden = hidden.to(device)
                cell = c_n.clone()
                cell = cell.to(device)     


            #optimizing:

            optimizer.zero_grad()
            batch_loss.backward(retain_graph=True)
            optimizer.step()

            no += 1

        print(loss/no)



In [12]:
data_input = data_train
for _ in range(20):
 train(data_input, hidden_size,vocab_size)

Streaming output truncated to the last 5000 lines.
tensor(3.3889, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.3889, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.3889, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.3918, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.3907, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.3925, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.3926, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.3929, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.3933, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.3930, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.3929, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.3921, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.3949, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.3924, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.3924, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.3952, device='cuda:0', grad_fn=<DivBackward0>)
tensor(3.3944, device='cuda:0', grad_fn=<DivBackward0

In [15]:
model_lstm.eval()

hidden = torch.zeros([1, 1, hidden_size])
hidden = hidden.to(device)

#making initial cell:
cell = torch.zeros([1, 1, hidden_size])
cell = cell.to(device)

start_idx = word_to_ix["the"]
print("The", end=" ")
input = torch.zeros([1, 1, vocab_size])   
input[0,0,start_idx] = 1 
input = input.to(device)

for i in range(15):
    predictions, h_n , c_n = model_lstm(input, hidden, cell)
    predictions = predictions.to(device)
    h_n = h_n.to(device)
    c_n = c_n.to(device)
    predictions = predictions.view((1, vocab_size)) #reshaping the predictions(originally [1,1,vocabb_size])
    predictions = nn.Softmax(dim=1)(predictions)
    idx = predictions.argmax()
    input = torch.zeros([1,1,vocab_size])
    input[0,0,idx] = 1
    input = input.to(device)
    print(ix_to_word[idx.item()], end=" ")
    hidden = h_n.clone()
    cell = c_n.clone()
    if idx == word_to_ix["."]:
        h_n = torch.zeors([h_n.shape])
        c_n =  torch.zeros([c_n.shape])

The how to help produce our new ebooks , how to help produce our new ebooks 